In [1]:
from src.sentinel_api import get_sentinel_api
from src.constant import GEOJSON_PATH, DOWNLOADED_DATA_PATH, REMOVE_ZIPS
import os

# get api 
api = get_sentinel_api()

In [2]:
# search by polygon, time, and Hub query keywords
products_query = api.query(
                    limit = 1,
                    filename = 'S2A_*',
                    producttype = 'S2MSI2Ap')

os.makedirs(DOWNLOADED_DATA_PATH, exist_ok = True) 
products = api.download_all(products_query, DOWNLOADED_DATA_PATH)
print(products)

Downloading: 100%|██████████| 1.13G/1.13G [00:57<00:00, 19.9MB/s]
MD5 checksumming: 100%|██████████| 1.13G/1.13G [00:33<00:00, 34.0MB/s](OrderedDict([('1826bfa9-b681-41d4-99f6-1d4cfd13774f', {'id': '1826bfa9-b681-41d4-99f6-1d4cfd13774f', 'title': 'S2A_MSIL2A_20180324T095031_N0206_R079_T34VFH_20180324T102144', 'size': 1133818164, 'md5': 'f6b85dc909fb657011b9dbb0cc175f58', 'date': datetime.datetime(2018, 3, 24, 9, 50, 31, 26000), 'footprint': 'POLYGON((24.40774485405625 56.4811775230197,24.386548238424314 56.44850855996068,24.294892845946208 56.306279986906574,24.203159228413103 56.164047032001285,24.112048384907432 56.02168886981792,24.021657027108123 55.879185132850886,23.983794298682664 55.819068255712935,22.59716193895971 55.84695640311729,22.638906428446163 56.833055285023114,24.435716349472024 56.79650515210518,24.40774485405625 56.4811775230197))', 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('1826bfa9-b681-41d4-99f6-1d4cfd13774f')/$value", 'Online': True, 'Creation

In [3]:
file_titles = []
zip_files = []

for product in products:
    for product_id in product:
        file_titles.append(product[product_id]['title'])
        zip_files.append(product[product_id]['path'])

        import zipfile
        archive = zipfile.ZipFile(product[product_id]['path'], 'r')
        archive.extractall(path=DOWNLOADED_DATA_PATH)

for zip_file in zip_files:
    if REMOVE_ZIPS:
        os.remove(zip_file)     

In [4]:
import rasterio
import rasterio.features
import rasterio.warp
import matplotlib.pyplot as plt

for file_title in file_titles:
    granule_path = os.path.join(DOWNLOADED_DATA_PATH, '{}.SAFE'.format(file_title), 'GRANULE')
    measurement_path = os.path.join(granule_path, os.listdir(granule_path)[0], 'IMG_DATA')
    for jp2_file_name in sorted(os.listdir(measurement_path)):
        print(jp2_file_name)
        jp2_file_path =  os.path.join(measurement_path, jp2_file_name)   

        with rasterio.open(jp2_file_path, "r", driver='JP2OpenJPEG') as dataset:
            # Read the dataset's data
            data = dataset.read(1)

            # Visualise the dataset's data
            fig, ax = plt.subplots()
            img = ax.imshow(data.real)
            plt.show()

R10m


RasterioIOError: './src/data/S2A_MSIL2A_20180324T095031_N0206_R079_T34VFH_20180324T102144.SAFE/GRANULE/L2A_T34VFH_A014376_20180324T095027/IMG_DATA/R10m' not recognized as a supported file format.